In [2]:
import geemap
import ee
from geemap import geojson_to_ee, ee_to_geojson


In [3]:
m = geemap.Map()
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [11]:
new_york= "data/nyc.geojson"

In [12]:
import geopandas as gpd

In [14]:
gdf= gpd.read_file(new_york)
gdf.head()

,boro_code,boro_name,shape_area,shape_leng,geometry
0,3,Brooklyn,1934142775.63,728147.130565,"MULTIPOLYGON (((-73.86327 40.58388, -73.86381 ..."
1,5,Staten Island,1623618684.31,325910.306061,"MULTIPOLYGON (((-74.05051 40.56642, -74.05047 ..."
2,1,Manhattan,636646082.483,360037.525777,"MULTIPOLYGON (((-74.01093 40.68449, -74.01193 ..."
3,2,Bronx,1187174772.32,463180.578878,"MULTIPOLYGON (((-73.89681 40.79581, -73.89694 ..."
4,4,Queens,3041418003.6,888197.010944,"MULTIPOLYGON (((-73.82645 40.59053, -73.82642 ..."


In [15]:
ee_fc = geojson_to_ee(new_york)
m.addLayer(ee_fc, {}, 'NYC')

In [16]:
m.centerObject(ee_fc)

In [20]:
ee.Initialize()

In [23]:
collection = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterDate("2022-01-01", "2022-12-31")
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 5))
)


In [24]:
image= collection.median()

In [25]:
vis_params = {
    "bands": ["B4", "B3", "B2"],
    "min": 0,
    "max": 2000,
}

In [26]:
subset = image.clip(ee_fc)

In [28]:
m.addLayer(subset, vis_params, "RGB")
m.centerObject(subset)
m

Map(bottom=556.0, center=[0.0, 0.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Se…

In [29]:
vis_params = {
    "bands": ["B8", "B4", "B3"],
    "min": 0,
    "max": 2000,
}
m.addLayer(subset, vis_params, "Near Infrared")

In [32]:
landcover= ee.ImageCollection("ESA/WorldCover/v100").select("Map").first()

In [33]:
subset_landcover = landcover.clip(ee_fc)

In [34]:
m.addLayer(subset_landcover, {}, "NYC Land Cover")
m

Map(bottom=98866.0, center=[40.707710007867334, -73.79791259765626], controls=(WidgetControl(options=['positio…

In [35]:
m.add_legend(builtin_legend="ESA_WorldCover")